# Protótipo da Pipeline dos Conjuntos de Dados — Schiphol Destinations

Este protótipo tem como objetivo validar o fluxo completo dos dados referente ao conjunto de dados Destinations da api do Schiphol.  
A pipeline foi construída para verificar se os valores estão sendo disponibilizados corretamente em cada etapa.

O foco principal é garantir **consistência**, **integridade** e **precisão** dos dados antes da implementação final da arquitetura.

In [1]:

import pandas as pd

df = pd.read_csv(r'../data/destinations.csv', index_col='Unnamed: 0')

dataframe_copy = df.copy()

pd.set_option('display.max_columns', None)   
pd.set_option('display.max_rows', None)      
pd.set_option('display.width', 300)         
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.float_format', '{:.2f}'.format)

# colunas a tratar
print(dataframe_copy.columns)
dataframe_copy

Index(['cidade', 'pais', 'codigo_iata', 'nome_publico'], dtype='object')


,cidade,pais,codigo_iata,nome_publico
0,Não Informado,French Polynes,AAA,"Anaa, Tu"
1,Não Informado,Australia,AAB,"Arrabury, Ql"
2,Al-Arish,Egypt,AAC,Al-Arish
3,Ad-Dabbah,Sudan,AAD,Ad-Dabbah
4,Annaba,Algeria,AAE,Annaba
5,Apalachicola,Usa,AAF,"Municipal, Fl"
6,Não Informado,Brazil,AAG,"Arapoti, Pr"
7,Aachen,Germany,AAH,Merzbrueck
8,Não Informado,Brazil,AAI,"Arraias, Td"
9,Awaradam,Surinam,AAJ,Cayana Airstrip


# Tratamento das Colunas do Dataset

Este documento descreve as ações previstas para tratar e padronizar as colunas:
`country`, `iata`, `publicName`, `city`

O foco é padronização, limpeza e validação.

---

## Checklist Geral (para todas as colunas)

- [x] Renomear colunas para português (`country` → `pais`, `iata` → `codigo_iata`, `publicName` → `nome_publico`, `city` → `cidade`)
- [x] Converter para minúsculas ou capitalizar
- [x] Remover espaços extras (início, fim e múltiplos no meio)
- [x] Padronizar nomes e formatos
- [x] Substituir valores inválidos ("N/A", "null", "none") por nulo real

---


# Renomeando

In [2]:
new_column_name = {
    'country': 'pais',
    'iata': 'codigo_iata',
    'publicName': 'nome_publico',
    'city': 'cidade'
}

dataframe_copy = dataframe_copy.rename(columns=new_column_name)
dataframe_copy

,cidade,pais,codigo_iata,nome_publico
0,Não Informado,French Polynes,AAA,"Anaa, Tu"
1,Não Informado,Australia,AAB,"Arrabury, Ql"
2,Al-Arish,Egypt,AAC,Al-Arish
3,Ad-Dabbah,Sudan,AAD,Ad-Dabbah
4,Annaba,Algeria,AAE,Annaba
5,Apalachicola,Usa,AAF,"Municipal, Fl"
6,Não Informado,Brazil,AAG,"Arapoti, Pr"
7,Aachen,Germany,AAH,Merzbrueck
8,Não Informado,Brazil,AAI,"Arraias, Td"
9,Awaradam,Surinam,AAJ,Cayana Airstrip


# Captalizando, removendo espaços em branco e deixando a primeira letra da palavra como maiúscula

In [3]:
dataframe_copy['pais'] = dataframe_copy['pais'].str.title()
dataframe_copy['pais'] = dataframe_copy['pais'].str.strip()

dataframe_copy['codigo_iata'] = dataframe_copy['codigo_iata'].str.upper()
dataframe_copy['codigo_iata'] = dataframe_copy['codigo_iata'].str.strip()

import ast

def get_english(v):
    if isinstance(v, dict):
        return v.get('english', '').title()
    if isinstance(v, str):
        try:
            d = ast.literal_eval(v)
            return d.get('english', '').title()
        except:
            return None
    return None

dataframe_copy['nome_publico'] = dataframe_copy['nome_publico'].apply(get_english)
dataframe_copy['nome_publico'] = dataframe_copy['nome_publico'].str.strip()

dataframe_copy['cidade'].str.title()
dataframe_copy['cidade'].str.strip()

0     Não Informado
1     Não Informado
2          Al-Arish
3         Ad-Dabbah
4            Annaba
5      Apalachicola
6     Não Informado
7            Aachen
8     Não Informado
9          Awaradam
10    Não Informado
11          Aalborg
12    Não Informado
13           Al Ain
14    Não Informado
15          Houston
16            Anapa
17           Aarhus
18    Não Informado
19    Não Informado
Name: cidade, dtype: object

# Verificando valores como ("N/A, "Null", "None")

In [4]:
dataframe_copy['pais'].isna().sum()
# np.int64(0)

dataframe_copy['pais'].isnull().sum()
# np.int64(0)

dataframe_copy['pais'].dtype
# dtype('O')

paises_unicos = ['French Polynes', 'Australia', 'Egypt', 'Sudan', 'Algeria', 'Usa',
       'Brazil', 'Germany', 'Surinam', 'Kiribati', 'Denmark',
       'South Africa', 'United Arab Emirates (The)', 'Venezuela',
       'Russia (Cis)', 'Indonesia', 'China']

dataframe_copy['pais'].isin(paises_unicos).value_counts()

# ------------------------------------------------------------------------------------

dataframe_copy['codigo_iata'].isna().sum()
# np.int64(0)

dataframe_copy['codigo_iata'].isnull().sum()
# np.int64(0)

dataframe_copy['codigo_iata'].dtype
# dtype('O')

codigo_unico = ['AAA', 'AAB', 'AAC', 'AAD', 'AAE', 'AAF', 'AAG', 'AAH', 'AAI',
       'AAJ', 'AAK', 'AAL', 'AAM', 'AAN', 'AAO', 'AAP', 'AAQ', 'AAR',
       'AAS', 'AAT']

dataframe_copy['codigo_iata'].isin(codigo_unico).value_counts()

# ------------------------------------------------------------------------------------

dataframe_copy['nome_publico'].isna().sum()
# np.int64(0)

dataframe_copy['nome_publico'].isnull().sum()
# np.int64(0)

dataframe_copy['nome_publico'].dtype
# dtype('O')

nomes_unicos = ['Anaa, Tu', 'Arrabury, Ql', 'Al-Arish', 'Ad-Dabbah', 'Annaba',
       'Municipal, Fl', 'Arapoti, Pr', 'Merzbrueck', 'Arraias, Td',
       'Cayana Airstrip', 'Aranuka', 'Aalborg', 'Malamala', 'Al Ain',
       'Anaco, An', 'Andrau Airpark, Tx', 'Vitiazevo', 'Aarhus',
       'Apalapsili', 'Altay']

dataframe_copy['nome_publico'].isin(nomes_unicos).value_counts()

# ------------------------------------------------------------------------------------

dataframe_copy['cidade'].isna().sum()
# np.int64(9)

dataframe_copy['cidade'].isnull().sum()
# np.int64(9)

dataframe_copy[dataframe_copy['cidade'].isna()]

dataframe_copy['cidade'] = dataframe_copy['cidade'].fillna('Não informado')

dataframe_copy['cidade'].isnull().sum()
# np.int64(0)

cidades_unica = ['Não informado', 'Al-Arish', 'Ad-Dabbah', 'Annaba', 'Apalachicola',
       'Aachen', 'Awaradam', 'Aalborg', 'Al Ain', 'Houston', 'Anapa',
       'Aarhus']

dataframe_copy['cidade'].isin(cidades_unica).value_counts()

cidade
True     11
False     9
Name: count, dtype: int64